In [ ]:
from neo4j.v1 import GraphDatabase, basic_auth 
driver = GraphDatabase.driver('bolt://localhost', auth=basic_auth('neo4j', 'test'))

import json
import os
from datetime import datetime

photo_base_dir = '/Users/rvanweverwijk/Pictures/2016'
thumbnail_base_dir = '/Users/rvanweverwijk/Pictures/thumbnails'

# watch("httpstream")


In [ ]:
session = driver.session()
session.run("MERGE (d:Directory {name:'" + photo_base_dir + "'})")
session.close()

In [ ]:
session = driver.session()
result = session.run('MATCH (d:Directory) RETURN d.name AS name')
for record in result:
    print(record["name"])
session.close()


In [ ]:
ignored_files = ['.DS_Store']

def get_current_time() :
    now = datetime.now()
    return int(now.strftime('%Y%m%d%H%M%S'))

def update_directory_structure(current_time):
    add_directories(current_time)
    delete_non_existing_directories(current_time)
    add_photos(current_time)
    delete_non_existing_photos(current_time)
    update_modification_times()
    tag_new_photos()

def create_directory_query(path, current_time):
    query = 'MERGE (root:Directory {name: "'+ photo_base_dir + '"})\nCREATE UNIQUE path = (root)'
    current_path = ''
    for dir in path:
        current_path = current_path + '/' + dir
        query = query + '-[:HAS_DIR]->(:Directory {name: "' + dir + '", fullPath: "' + current_path + '"})'
    query = query + '\nwith nodes(path) as dirs\nunwind dirs as dir\nset dir._lastModified = toInt(' + str(current_time) + ')'
    return query

def create_photo_query():
    query = '''
            MATCH (d:Directory {fullPath: {fullDirectoryPath}})\n
            MERGE (p:Photo {name:{fileName}, directory: {fullDirectoryPath}})\n
            set p._lastModified = {currentTime}\n
            MERGE (d)-[:CONTAINS_PHOTO]->(p)
    '''
    return query

def update_modification_times():
    print('update modification times')
    session = driver.session()
    session.run('match (d:Directory) set d._timesModified = coalesce(d._timesModified, 0) + 1')
    session.run('match (p:Photo) set p._timesModified = coalesce(p._timesModified, 0) + 1')
    session.close()
    
def add_directories(current_time):
    print('add directories({0})'.format(current_time))
    session = driver.session()
    with session.begin_transaction() as tx:
        for root, dirs, files in os.walk(photo_base_dir):
            if not dirs:
                path = root.replace(photo_base_dir + '/', '').split('/')
                tx.run(create_directory_query(path, current_time))
        tx.success = True
    session.close()
        
def add_photos(current_time):
    print('add photos({0})'.format(current_time))
    session = driver.session()
    with session.begin_transaction() as tx:
        for root, dirs, files in os.walk(photo_base_dir):
            path = root.replace(photo_base_dir, '')
            for file in files: 
                if (file not in ignored_files):
                    result = tx.run(statement=create_photo_query(),parameters={'currentTime': current_time, 'fullDirectoryPath': path, 'fileName': file})
        tx.success = True
    session.close()
    
def tag_new_photos():
    print('tag new photos')
    session = driver.session()
    result = session.run('''match (p:Photo {_timesModified: 1})
        with p, filter(tag in split(p.directory, "/") where tag <> "") as tags
        unwind tags as tag
        MERGE (t:Tag {name: tag})
        create (p)-[:HAS_TAG]->(t)''')
    sum = result.consume().counters
    print(sum)
    session.close()

def delete_non_existing_directories(current_time) :
    print('delete directories({0})'.format(current_time))
    session = driver.session()
    with session.begin_transaction() as tx:
        result = tx.run(statement='''match (d:Directory)
            where d._lastModified < {currentTime}
            with d
            optional match (d)-[:CONTAINS_PHOTO]->(p:Photo)
            detach delete d,p''', parameters={'currentTime': current_time})
        sum = result.consume().counters
        print(sum)
        tx.success = True
    session.close

def delete_non_existing_photos(current_time) :
    print('delete photos({0})'.format(current_time))
    session = driver.session()
    with session.begin_transaction() as tx:
        result = tx.run(statement='match (p:Photo) where p._lastModified < {currentTime} detach delete p', parameters={'currentTime': current_time})
        sum = result.consume().counters
        print(sum)
        tx.success = True
    session.close

In [ ]:
current_time = get_current_time()
update_directory_structure(current_time)

In [ ]:
current_time

## resize image

In [ ]:
flickrSize = {
  8: {'name': 's', 'size': (75, 75)},
  7: {'name': 't', 'size': (100, 100)},
  6: {'name': 'q', 'size': (150, 150)},
  5: {'name': 'm', 'size': (240, 240)},
  4: {'name': 'n', 'size': (320, 320)},
  3: {'name': 'z', 'size': (640, 640)},
  2: {'name': 'c', 'size': (800, 800)},
  1: {'name': 'b', 'size': (1024, 1024)}
}

from PIL import Image
import glob, os

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def make_thumbnails(local_path):
    print local_path
    create_directory(os.path.dirname(thumbnail_base_dir + local_path))
    im = Image.open(photo_base_dir + local_path)
    for key in sorted(flickrSize):
        file, ext = os.path.splitext(local_path)
        thumbnail_filename = thumbnail_base_dir + file + '_' + flickrSize[key]['name'] + ext
        if (os.path.exists(thumbnail_filename)):
            break;
        im.thumbnail(flickrSize[key]['size'])
        im.save(thumbnail_filename, "JPEG")

In [ ]:
for infile in glob.glob(photo_base_dir + '/uitzoeken/*.JPG'):
    make_thumbnails(infile.replace(photo_base_dir, ''))

In [ ]:
session = driver.session()
with session.begin_transaction() as tx:
    result = tx.run(statement='''match (p:Photo) where not EXISTS(p.thumbnail) return id(p) as photoId, p.directory + '/' + p.name as local_path limit 10000''')
    for row in result:
        local_path = row['local_path']
        photo_id = row['photoId']
        file, ext = os.path.splitext(local_path)
        if (ext == '.JPG'):
            make_thumbnails(local_path)
            update_result = tx.run(statement='''match (p:Photo) where id(p) = {photoId} set p.thumbnail = true''', parameters={'photoId': photo_id})
#             sum = update_result.consume().counters
#             print(sum)
    tx.success = True
session.close()